In [94]:
import requests
import re
import time
import random
import json
import os
cong_key=''

In [34]:
cong={}
next='https://api.congress.gov/v3/member?'
while True:
    member_resp=requests.get(f'{next}&api_key={cong_key}')
    for m in member_resp.json()['members']:
        cong[re.sub(r'[^a-zA-Z ]','',m['name'].lower()).strip()]=m['bioguideId']
    if not member_resp.json().get('pagination'):
        break
    next=member_resp.json().get('pagination').get('next')
    time.sleep(random.random()*2)

MissingSchema: Invalid URL 'None&api_key=Jq9Uy2kgbNouvTJfoixzEYYQQJ88FAjM3D5HFPne': No scheme supplied. Perhaps you meant https://None&api_key=Jq9Uy2kgbNouvTJfoixzEYYQQJ88FAjM3D5HFPne?

In [29]:
with open('./opensecrets_info.json', 'r') as f:
    cong_info=json.load(f)

In [67]:
from string import ascii_lowercase as alc
mapping={}
manual=[]
for o_id,info in cong_info.items():
    name=re.sub(r'[^a-zA-Z ]','',info['name'].lower()).strip()
    f,*n=name.split()
    if len(n)==1:
        new=" ".join([n[-1],f])
    elif len(n)==2:
        new=" ".join([n[-1],f,n[0]])

    #guess cycle
    if not cong.get(new):
        new+='  '
        for let in alc:
            new=new[:-1]+let
            if cong.get(new):break
    mapping[o_id]={'id':cong.get(new),'name':info['name']}
    if not cong.get(new):
        manual.append(info['name'])

In [73]:
usable={k:v for k,v in mapping.items() if v['id']}
new_info={v['id']:cong_info[k]['industry'] for k,v in usable.items()}

In [93]:
bill_id=set()
bills=[]
for file in os.listdir('./votes'):
    with open(f'./votes/{file}') as f:
        for z in json.load(f):
            if z[0] not in bill_id:
                bill_id.add(z[0])
                bills.append(z)

In [109]:
new_info={k:v for k,v in new_info.items() if v}
with open('./gov_id_industry.json', 'w') as f:
    json.dump(new_info,f)

In [184]:
term_to_industry={}
for id, info in new_info.items():
    for i in info:
        for term in cleaner(i['industry_name']):
            if not term_to_industry.get(term):
                term_to_industry[term]=set([i['industry_name']])
            else:
                term_to_industry[term].add(i['industry_name'])

In [185]:
term_to_industry

{'retired': {'Retired'},
 'estate': {'Real Estate'},
 'real': {'Real Estate'},
 'leadership': {'Leadership PACs'},
 'pacs': {'Leadership PACs'},
 'casinos': {'Casinos/Gambling'},
 'gambling': {'Casinos/Gambling'},
 'defense': {'Defense Aerospace', 'Defense Electronics', 'Misc Defense'},
 'misc': {'Misc Defense',
  'Misc Finance',
  'Misc Issues',
  'Misc Manufacturing &amp; Distributing',
  'Misc Transport',
  'Misc Unions'},
 'production': {'Crop Production &amp; Basic Processing'},
 'crop': {'Crop Production &amp; Basic Processing'},
 'processing': {'Crop Production &amp; Basic Processing',
  'Food Processing &amp; Sales'},
 'amp': {'Beer, Wine &amp; Liquor',
  'Building Materials &amp; Equipment',
  'Crop Production &amp; Basic Processing',
  'Electronics Mfg &amp; Equip',
  'Fisheries &amp; Wildlife',
  'Food &amp; Beverage',
  'Food Processing &amp; Sales',
  'Forestry &amp; Forest Products',
  'Misc Manufacturing &amp; Distributing',
  'Oil &amp; Gas',
  'Poultry &amp; Eggs',
  '

In [206]:
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#score, total tuple for each industry
lobbying_scores={id:{i['industry_name']:[0,0] for i in info} for id, info in new_info.items()}
#vader
sid = SentimentIntensityAnalyzer()
stopwords = set(stopwords.words('english'))
cleaner = lambda sentence: set(re.sub(r'[^a-zA-Z ]',' ',sentence.lower()).split()) - stopwords
neg_votes=set(['Nay','No'])
pos_votes=set(['Aye','Yea'])

#There are only 26 bills with substantial terms that match up to industry names.
for bill in bills:
    #determine sentiment
    sentence=tokenize.sent_tokenize(bill[3])[0]
    ss = sid.polarity_scores(sentence)
    #1 for positive, 0 for negative
    max_sentiment=-1
    if ss['pos']>ss['neg']:max_sentiment=1

    #bill terms
    votes=bill[-1] if bill[-1] else bill[-2]
    relevant_terms=cleaner(bill[1])

    
    #candidate loop
    for id, info in new_info.items():
        #candidate industries:
        relevant_industries=set()
        relevant_industries.update(sum([list(cleaner(i['industry_name'])) for i in info],[]))
    
        #relevant?:
        if relevant_terms & relevant_industries:
            #scores move in intervals of 1
            cast=votes.get(id)
            if cast in pos_votes:adjust=1*max_sentiment
            elif cast in neg_votes:adjust=-1*max_sentiment
            for term in relevant_terms & relevant_industries:
                for industry in term_to_industry[term]:
                    if lobbying_scores[id].get(industry):
                        lobbying_scores[id][industry][0]+=max(adjust,0) #-1 -> 0
                        lobbying_scores[id][industry][1]+=1 #total        

In [207]:
lobbying_scores

{'C001054': {'Retired': [0, 0],
  'Real Estate': [0, 0],
  'Leadership PACs': [0, 1],
  'Casinos/Gambling': [0, 0],
  'Misc Defense': [2, 3],
  'Crop Production &amp; Basic Processing': [0, 0],
  'Lawyers/Law Firms': [0, 0],
  'Lobbyists': [0, 0],
  'Building Materials &amp; Equipment': [1, 1],
  'Insurance': [0, 0]},
 'M001212': {'Crop Production &amp; Basic Processing': [0, 0],
  'Retired': [0, 0],
  'Real Estate': [0, 0],
  'Leadership PACs': [0, 1],
  'Republican/Conservative': [0, 0],
  'Building Materials &amp; Equipment': [1, 1],
  'General Contractors': [0, 0],
  'Misc Defense': [2, 3],
  'Health Professionals': [1, 2],
  'Agricultural Services/Products': [1, 1]},
 'R000575': {'Misc Defense': [2, 3],
  'Defense Aerospace': [2, 3],
  'Lobbyists': [0, 0],
  'Leadership PACs': [0, 1],
  'Defense Electronics': [2, 3],
  'Lawyers/Law Firms': [0, 0],
  'Real Estate': [0, 0],
  'Crop Production &amp; Basic Processing': [0, 0],
  'Insurance': [0, 0],
  'General Contractors': [0, 0]},
 